In [1]:
using PyPlot

In [2]:
function Heun(f, t0, y0, h, N)
    t = zeros(1, N + 1)
    y = zeros(1, N + 1)
    # Initial values:
    t[1] = t0
    y[1] = y0
    for i = 1:N
        t[i+1] = t[i] + h
        y[i+1] = y[i] + h * (f(t[i] , y[i]) + f(t[i] + h , y[i] + h * f(t[i], y[i]))) / 2.
    end
    return (t[N + 1], y[N + 1])
end

Heun (generic function with 1 method)

In [ ]:
t0 = -3.
T = 6.
y0 = 1./901.

y_analy(t) = 1. ./ (1 + 100 * t.^2)
f(t, x) = -200. * t * x^2

m = 2
K = 10
h0 = 1e-2
epsilon = 1e-5

t = []
y = [] 
func_count = 0; # counts number of function evaluations

push!(t, t0);
push!(y, y0);

i = 1
h = h0;
H = 2.*h

while (t[i] < T + t0)
    tn = t[i] + H
    tnh, ynh = Heun(f, t[i], y[i], H, 1)
    tnh2, ynh2 = Heun(f, t[i], y[i], H/2, 2)
    func_count = func_count + 9;
    τ = (ynh2 - ynh) / (H ^(m+1) * (1. - 2.0^(-m)))
    htilde = (epsilon / (K * T * norm(τ)))^(1.0/m)
    
    if (htilde > 0.25*H)
        push!(t, tn)
        newy = ((2.0^m)*ynh2 - ynh) / (2.0^m - 1);
        push!(y, newy)
        i = i + 1;
        h = h0;
        H = 2. * h
    else
        H = 2.*htilde;
    end
end

In [ ]:
global_err = maximum(abs.(y - y_analy(t)))
println("Global error: ", global_err)
println("Function evaluations: ", func_count)

In [ ]:
figure(figsize=(4,4))
    plot(t[:], y[:], label="Adaptive Heun")
    plot(t[:], y_analy(t[:]), label="Analytic")
    legend();

In [ ]:
function Heunh(f, t0, y0, h, T)
    N = convert(Int64, floor(T/h))
    t = zeros(1, N + 1)
    y = zeros(1, N + 1)
    t[1] = t0
    y[1] = y0
    
    for i = 1:N
        t[i+1] = t[i] + h
        y[i+1] = y[i] + h * (f(t[i], y[i]) + f(t[i] + h, y[i] + h * f(t[i], y[i])))
    end
    return (t,y)
end

In [ ]:
hmin = -5
hmax = -2
harray = logspace(hmin, hmax, 100)
d = length(array)
const_global_err = zeros(1,d)

for i = 1:d
    h = harray[i];
    th, yh = Heunh(f, t0, y0, h, T)
    const_global_err[i] = maximum(abs.(yh - y_analy(th)))
end

equiv_h = minimum(abs.(const_global_err[:] - global_err));
N = convert(Int64, floor(T/equiv_h))
func_evals = 3*N;

println("Comparable h: ", equiv_h)
println("Function evaluations: ", func_evals)